# Prepare Experiment & Deepspeed config (**MANDATORY**)
***

In [ ]:
ds_config = {
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "optimizer": {
        "type": "AdamW",
        "params": {
            "lr": "auto",
            "betas": "auto",
            "eps": "auto",
            "weight_decay": "auto"
        }
    },

    "zero_optimization": {
        "stage": 2,
        "allgather_partitions": True,
        "allgather_bucket_size": 5e8,
        "overlap_comm": True,
        "reduce_scatter": True,
        "reduce_bucket_size": 5e8,
        "contiguous_gradients": True,
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 200,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": False
}

training_args = {
    "do_train": False,
    "do_eval": True,
    "do_predict": True,
    "num_train_epochs": 4,
    "gradient_accumulation_steps": 1,
    "per_device_train_batch_size": 64,
    "per_device_eval_batch_size": 32,
    "fp16": True,
    "weight_decay": 0.0,
    "warmup_steps": 0,
    "learning_rate": 5e-5,
    "logging_strategy": "epoch",
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "save_total_limit": 1,
    "load_best_model_at_end": True,
    "metric_for_best_model": "eval_accuracy",
    "greater_is_better": True,
}

model_args = {
}

# usually overriden by external config:
num_gpus = 1
model_name ="EleutherAI/gpt-neo-2.7B"
logdir = "data/models/tests/"
override_logdir = True

dataset = "tweet-eval"
seed = 8197

load_pretrained_weights = True
from_checkpoint = "data/models/sequence_classification/tweet-eval/EleutherAI/gpt-neo-2.7B/bs32_lr_5e-05/checkpoint-846"
deepspeed = False

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
if not deepspeed:
    pass
    #os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import numpy as np
import pandas as pd
from datasets import load_dataset
import time
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
import datasets
from transformers import Trainer, TrainingArguments
import fastmodellib as fml

pd.set_option('display.max_colwidth', 400)

In [ ]:
training_args = TrainingArguments(
    output_dir=logdir,
    overwrite_output_dir=override_logdir,
    logging_dir=logdir,
    deepspeed= logdir + "/ds_config.json" if deepspeed else None,
    report_to="tensorboard",
    **training_args
)

# Tokenize the dataset
***

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from datasets import load_dataset

def load_action_norm_split(path):
    # loads the moral stories like data under path
    train, dev, test = [pd.read_json(f"{path}{x}.jsonl", lines=True) for x in ["train", "dev", "test"]]

    # construct dataframes that can actually be used
    assign_action = lambda x: x["moral_action"] if x["label"] == 1 else x["immoral_action"]
    train["action"] = train.apply(assign_action, axis=1)
    dev["action"] = dev.apply(assign_action, axis=1)
    test["action"] = test.apply(assign_action, axis=1)

    subset = ["norm", "action", "label"]
    train = train[subset]
    dev = dev[subset]
    test = test[subset]
    return train, dev, test

if dataset=="moral-stories" or dataset == "contrastive-moral-stories":
    if dataset == "moral-stories":
        dataset_folder = "data/contrastive_moral_stories/original_ms/action+norm/norm_distance/"
    else:
        dataset_folder = "data/contrastive_moral_stories/anti_ms_llama/action+norm/norm_distance/"

    train, dev, test = load_action_norm_split(dataset_folder)

    train, dev, test = load_action_norm_split(dataset_folder)

    data = datasets.DatasetDict()
    data["train"] = datasets.Dataset.from_pandas(train)
    data["dev"] = datasets.Dataset.from_pandas(dev)
    data["test"] = datasets.Dataset.from_pandas(test)

    def tokenize(samples):
        return tokenizer(samples["action"], samples["norm"], truncation=True, padding=False)

    tokenized_data = data.map(tokenize, batched=True, batch_size=1000).shuffle(seed=seed)

elif dataset=="tweet-eval":
    te = load_dataset("tweet_eval", "hate")
    te["dev"] = te.pop("validation")
    
    def tokenize(samples):
        return tokenizer(samples["text"], padding=False)

    tokenized_data = te.map(tokenize, batched=True, batch_size=1000).shuffle(seed=seed)

elif dataset=="swag":
    from sklearn.model_selection import train_test_split

    swag = load_dataset("swag", "regular")

    def tokenize(data):
        data["original_label"] = data["label"]
        data["label"] = [int(x in {0,1}) for x in data["original_label"]]
        # if the correct answer is in the first two options, we use the sample as a positive one
        # if not, then we use the very first option of the sample (always an incorrect option!) as a false sample
        endings = []
        for i in range(len(data["label"])):
            ol = data["original_label"][i]
            ending = data[f"ending{ol}"][i] if ol in {0,1} else data["ending0"][i]
            endings.append(ending)
        data["ending"] = endings
        
        return tokenizer(data["startphrase"], data["ending"], padding=False)

    train = swag["train"].to_pandas()
    test = swag["validation"].to_pandas()
    dev, test = train_test_split(test, test_size=0.5, shuffle=False, random_state=seed)

    data = datasets.DatasetDict()
    data["train"] = datasets.Dataset.from_pandas(train)
    data["dev"] = datasets.Dataset.from_pandas(dev)
    data["test"] = datasets.Dataset.from_pandas(test)

    tokenized_data = data.map(tokenize, batched=True, batch_size=1000)

elif dataset=="boolq":

    boolq = load_dataset("boolq").shuffle(seed=seed)
    train = boolq["train"].to_pandas()
    dev = boolq["validation"].to_pandas()
    # split into dev and test
    n = len(dev)//2
    test = dev[n:]
    dev = dev[:n]

elif dataset=="rte":
    from datasets import load_dataset
else:
    raise ValueError(f"Unknown task '{dataset}'")

# Load the model

In [ ]:
model = fml.load_model(model_name=model_name, from_checkpoint=from_checkpoint, load_pretrained_weights=load_pretrained_weights,
                       model_class=AutoModelForSequenceClassification, **model_args)

In [ ]:
# set padding token if necessary
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id

# Prepare Trainer
***

In [ ]:
from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc =  metric.compute(predictions=predictions, references=labels)
    return acc

In [ ]:
from transformers import DataCollatorWithPadding

dc = DataCollatorWithPadding(tokenizer, padding=True, pad_to_multiple_of=8, return_tensors="pt")

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=dc,
    args=training_args,
    train_dataset=tokenized_data["train"] if training_args.do_train else None,
    eval_dataset=tokenized_data["dev"] if training_args.do_eval else None,
    compute_metrics=compute_metrics
)

In [ ]:
if training_args.do_train:
    trainer.train()

In [ ]:
if training_args.do_predict:
    print("RUNNING TESTS")
    for split, data in tokenized_data.items():
        print(split)
        r = trainer.evaluate(data, metric_key_prefix=f"test_{split}")
        print(r)